In [18]:
"""
Created on Wed Feb 26 17:01:56 2020

"""

""" IMPORTS """
import sys
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.constraints import *
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D

import h5py
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
from lambda_layers import *
from binary_ops import *
%matplotlib qt

In [ ]:
""" FUNCTIION AND VARIABLE DEFINITIONS """
def binary_tanh(x):
    return binary_tanh_op(x)

H = 1.
kernel_lr_multiplier = 'Glorot'

# # nn
batch_size = 50
epochs = 20
channels = 1
img_rows = 30
img_cols = 30
filters = 32
kernel_size = (32, 32)
pool_size = (2, 2)
hidden_units = 128
classes = 10
use_bias = False

# # learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# # BN
epsilon = 1e-6
momentum = 0.9

# # dropout
p1 = 0.25
p2 = 0.5

hdf5_dir = Path("../data/hdf5/")

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read
        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images= []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_vids.h5", "r+")

    images = np.array(file["/images"]).astype("float32")

    return images

def np_streak(x):
    input_dims = np.shape(x)
    output_shape = (input_dims[0],input_dims[1],input_dims[1]+input_dims[2],input_dims[3],input_dims[4])
    streak_tensor = np.zeros(output_shape)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            streak_tensor[i,j,j:(output_shape[3]+j),:,:] = x[i,j,:,:,:]
    #return streak_tensor
    return np.sum(streak_tensor,axis=1)

def mask(val,ims,mask):
    for i in range(np.shape(val)[0]):
        for j in range(np.shape(val)[1]):
            val[i,j,:,:] = ims[i,j,:,:] * mask
    return val



ims = read_many_hdf5(784)
# ims = np.ones((3943,30,32,32,1))
ims = np.reshape(ims, (-1,30,32,32,1))
ims = ims[:750]
# temp = np.zeros((1,32,32,1))
validate2 = np.zeros((750,30,32,32,1))
bk_temp = np.random.randint(0,2,(1,32,32,1))
validate2 = mask(validate2,ims,bk_temp)
ims2 = np_streak(validate2)


validate = ims

validate = validate / 255
ims2 = ims2 /255
ims = ims/255
#X_train, X_test, y_train, y_test = train_test_split(ims, validate, test_size=(1/3), random_state=42)
X_train, X_test, y_train, y_test = train_test_split(ims2, validate, test_size=(1/3), random_state=42)

MX_train, MX_test, My_train, My_test = train_test_split(ims,ims, test_size = 1/3, random_state = 42)

print(np.shape(X_test))
print(np.shape(X_train))

reduce_lr = ReduceLROnPlateau(monitor='val_loss',verbose=1, factor=0.5,
                              patience=50, min_lr=0.000001)
early_stopping = EarlyStopping(patience=90,verbose=1,restore_best_weights=True)   


def custom_loss(y_true, y_pred):

  ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
  mse_loss = K.mean(K.square(y_pred-y_true))
  #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

  ssim_loss = 0.5*ssim_loss
  mse_loss = 0.5*mse_loss

  return ssim_loss + mse_loss

def ssim_loss(y_true,y_pred):  
    return (1.0-tf.image.ssim(y_true,y_pred,1))/2.0

def mse_loss(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true))



In [20]:
""" VIDEO FUNCTIONS FOR CHECKING POST TRAINING"""

def get_mask(model,l=0):
    b = binarize(model.layers[l].weights[0])
    figb,axb = plt.subplots(1,1)
    axb.imshow(np.reshape(b,(32,32)),cmap="gray")
    
def show_video(y_pred,y_true,num):
    yp = np.reshape(y_pred,(-1,30,32,32))
    yt = np.reshape(y_true,(-1,30,32,32))
    fig,ax = plt.subplots(nrows=5,ncols=6,sharex=True,sharey=True)
    fig2,ax2 = plt.subplots(nrows=5,ncols=6,sharex=True,sharey=True)
    for row in range(5):
        for col in range(6):
            ax[row,col].imshow(yp[num][5*row+col],cmap="gray")
            ax2[row,col].imshow(yt[num][5*row+col],cmap="gray")

def show_all_videos(videos,rows,cols):
    yp = np.reshape(videos, (-1,30,32,32))
    fix3,ax3 = plt.subplots(nrows=rows, ncols = cols)
    for row in range(rows):
        for col in range(cols):
            ax3[row,col].imshow(yp[rows*row+col][3],cmap="gray")


In [31]:
""" FORWARD MODEL """

forward_model = Sequential()
forward_model.add(Input(shape=(30,32,32,1),batch_size = 50))
forward_model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1')))
forward_model.add(Lambda(streak,output_shape=streak_output_shape))
forward_model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
forward_model.add(Flatten())
forward_model.add(Dense(30720, activation = 'relu'))
forward_model.add(Reshape((30,32,32,1)))
forward_model.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = ['mean_squared_error'])
forward_model.summary()
forward_history = forward_model.fit(MX_train, My_train,
          batch_size = 50,epochs= 500,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr, early_stopping])



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_88 (TimeDis (50, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_14 (Lambda)           (50, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_15 (Lambda)           (50, 62, 32, 1)           0         
_________________________________________________________________
flatten_7 (Flatten)          (50, 1984)                0         
_________________________________________________________________
dense_7 (Dense)              (50, 30720)               60979200  
_________________________________________________________________
reshape_7 (Reshape)          (50, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Epoch 59/500
500/500 - 2s - loss: 0.2166 - mean_squared_error: 0.1125 - val_loss: 0.2231 - val_mean_squared_error: 0.1136
Epoch 60/500
500/500 - 2s - loss: 0.2152 - mean_squared_error: 0.1108 - val_loss: 0.2252 - val_mean_squared_error: 0.1165
Epoch 61/500
500/500 - 2s - loss: 0.2161 - mean_squared_error: 0.1120 - val_loss: 0.2229 - val_mean_squared_error: 0.1135
Epoch 62/500
500/500 - 2s - loss: 0.2158 - mean_squared_error: 0.1119 - val_loss: 0.2232 - val_mean_squared_error: 0.1139
Epoch 63/500
500/500 - 2s - loss: 0.2148 - mean_squared_error: 0.1106 - val_loss: 0.2227 - val_mean_squared_error: 0.1133
Epoch 64/500
500/500 - 2s - loss: 0.2151 - mean_squared_error: 0.1110 - val_loss: 0.2219 - val_mean_squared_error: 0.1123
Epoch 65/500
500/500 - 2s - loss: 0.2139 - mean_squared_error: 0.1097 - val_loss: 0.2247 - val_mean_squared_error: 0.1157
Epoch 66/500
500/500 - 2s - loss: 0.2149 - mean_squared_error: 0.1111 - val_loss: 0.2216 - val_mean_squared_error: 0.1116
Epoch 67/500
500/500 - 2

Epoch 126/500
500/500 - 2s - loss: 0.2036 - mean_squared_error: 0.1017 - val_loss: 0.2193 - val_mean_squared_error: 0.1095
Epoch 127/500
500/500 - 2s - loss: 0.2050 - mean_squared_error: 0.1036 - val_loss: 0.2187 - val_mean_squared_error: 0.1088
Epoch 128/500
500/500 - 2s - loss: 0.2035 - mean_squared_error: 0.1018 - val_loss: 0.2160 - val_mean_squared_error: 0.1053
Epoch 129/500
500/500 - 2s - loss: 0.2027 - mean_squared_error: 0.1009 - val_loss: 0.2186 - val_mean_squared_error: 0.1086
Epoch 130/500
500/500 - 2s - loss: 0.2051 - mean_squared_error: 0.1038 - val_loss: 0.2184 - val_mean_squared_error: 0.1082
Epoch 131/500
500/500 - 2s - loss: 0.2056 - mean_squared_error: 0.1047 - val_loss: 0.2162 - val_mean_squared_error: 0.1056
Epoch 132/500
500/500 - 2s - loss: 0.2043 - mean_squared_error: 0.1029 - val_loss: 0.2221 - val_mean_squared_error: 0.1135
Epoch 133/500
500/500 - 2s - loss: 0.2046 - mean_squared_error: 0.1033 - val_loss: 0.2186 - val_mean_squared_error: 0.1088
Epoch 134/500
50

Epoch 193/500
500/500 - 2s - loss: 0.1961 - mean_squared_error: 0.0965 - val_loss: 0.2148 - val_mean_squared_error: 0.1039
Epoch 194/500
500/500 - 2s - loss: 0.1965 - mean_squared_error: 0.0967 - val_loss: 0.2174 - val_mean_squared_error: 0.1074
Epoch 195/500
500/500 - 2s - loss: 0.1981 - mean_squared_error: 0.0988 - val_loss: 0.2164 - val_mean_squared_error: 0.1061
Epoch 196/500
500/500 - 2s - loss: 0.1963 - mean_squared_error: 0.0968 - val_loss: 0.2193 - val_mean_squared_error: 0.1101
Epoch 197/500
500/500 - 2s - loss: 0.1979 - mean_squared_error: 0.0986 - val_loss: 0.2152 - val_mean_squared_error: 0.1047
Epoch 198/500
500/500 - 2s - loss: 0.1961 - mean_squared_error: 0.0966 - val_loss: 0.2135 - val_mean_squared_error: 0.1024
Epoch 199/500
500/500 - 2s - loss: 0.1956 - mean_squared_error: 0.0961 - val_loss: 0.2144 - val_mean_squared_error: 0.1035
Epoch 200/500
500/500 - 2s - loss: 0.1975 - mean_squared_error: 0.0982 - val_loss: 0.2140 - val_mean_squared_error: 0.1030
Epoch 201/500
50

Epoch 259/500
500/500 - 2s - loss: 0.1856 - mean_squared_error: 0.0886 - val_loss: 0.2095 - val_mean_squared_error: 0.0978
Epoch 260/500
500/500 - 2s - loss: 0.1855 - mean_squared_error: 0.0886 - val_loss: 0.2096 - val_mean_squared_error: 0.0980
Epoch 261/500
500/500 - 2s - loss: 0.1856 - mean_squared_error: 0.0887 - val_loss: 0.2098 - val_mean_squared_error: 0.0982
Epoch 262/500
500/500 - 2s - loss: 0.1855 - mean_squared_error: 0.0886 - val_loss: 0.2097 - val_mean_squared_error: 0.0980
Epoch 263/500
500/500 - 2s - loss: 0.1855 - mean_squared_error: 0.0886 - val_loss: 0.2096 - val_mean_squared_error: 0.0981
Epoch 264/500
500/500 - 2s - loss: 0.1855 - mean_squared_error: 0.0886 - val_loss: 0.2094 - val_mean_squared_error: 0.0978
Epoch 265/500
500/500 - 2s - loss: 0.1855 - mean_squared_error: 0.0887 - val_loss: 0.2097 - val_mean_squared_error: 0.0981
Epoch 266/500
500/500 - 2s - loss: 0.1853 - mean_squared_error: 0.0885 - val_loss: 0.2095 - val_mean_squared_error: 0.0979
Epoch 267/500
50

Epoch 325/500
500/500 - 2s - loss: 0.1831 - mean_squared_error: 0.0873 - val_loss: 0.2085 - val_mean_squared_error: 0.0969
Epoch 326/500
500/500 - 2s - loss: 0.1831 - mean_squared_error: 0.0873 - val_loss: 0.2086 - val_mean_squared_error: 0.0970
Epoch 327/500
500/500 - 2s - loss: 0.1831 - mean_squared_error: 0.0872 - val_loss: 0.2085 - val_mean_squared_error: 0.0969
Epoch 328/500
500/500 - 2s - loss: 0.1831 - mean_squared_error: 0.0872 - val_loss: 0.2085 - val_mean_squared_error: 0.0969
Epoch 329/500
500/500 - 2s - loss: 0.1831 - mean_squared_error: 0.0872 - val_loss: 0.2086 - val_mean_squared_error: 0.0970
Epoch 330/500
500/500 - 2s - loss: 0.1830 - mean_squared_error: 0.0872 - val_loss: 0.2085 - val_mean_squared_error: 0.0969
Epoch 331/500
500/500 - 2s - loss: 0.1830 - mean_squared_error: 0.0872 - val_loss: 0.2086 - val_mean_squared_error: 0.0970
Epoch 332/500
500/500 - 2s - loss: 0.1830 - mean_squared_error: 0.0872 - val_loss: 0.2084 - val_mean_squared_error: 0.0968
Epoch 333/500
50

Epoch 391/500
500/500 - 2s - loss: 0.1822 - mean_squared_error: 0.0867 - val_loss: 0.2082 - val_mean_squared_error: 0.0966
Epoch 392/500
500/500 - 2s - loss: 0.1822 - mean_squared_error: 0.0867 - val_loss: 0.2083 - val_mean_squared_error: 0.0967
Epoch 393/500
500/500 - 2s - loss: 0.1822 - mean_squared_error: 0.0867 - val_loss: 0.2082 - val_mean_squared_error: 0.0966
Epoch 394/500
500/500 - 2s - loss: 0.1822 - mean_squared_error: 0.0867 - val_loss: 0.2083 - val_mean_squared_error: 0.0966
Epoch 395/500
500/500 - 2s - loss: 0.1822 - mean_squared_error: 0.0867 - val_loss: 0.2081 - val_mean_squared_error: 0.0965
Epoch 396/500
500/500 - 2s - loss: 0.1821 - mean_squared_error: 0.0867 - val_loss: 0.2083 - val_mean_squared_error: 0.0967
Epoch 397/500
500/500 - 2s - loss: 0.1821 - mean_squared_error: 0.0867 - val_loss: 0.2083 - val_mean_squared_error: 0.0966
Epoch 398/500
500/500 - 2s - loss: 0.1822 - mean_squared_error: 0.0867 - val_loss: 0.2082 - val_mean_squared_error: 0.0965
Epoch 399/500
50

Epoch 457/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0865 - val_loss: 0.2081 - val_mean_squared_error: 0.0963
Epoch 458/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0864 - val_loss: 0.2081 - val_mean_squared_error: 0.0964
Epoch 459/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0864 - val_loss: 0.2081 - val_mean_squared_error: 0.0964
Epoch 460/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0865 - val_loss: 0.2080 - val_mean_squared_error: 0.0963
Epoch 461/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0864 - val_loss: 0.2081 - val_mean_squared_error: 0.0964
Epoch 462/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0864 - val_loss: 0.2082 - val_mean_squared_error: 0.0965
Epoch 463/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0864 - val_loss: 0.2080 - val_mean_squared_error: 0.0964
Epoch 464/500
500/500 - 2s - loss: 0.1817 - mean_squared_error: 0.0864 - val_loss: 0.2081 - val_mean_squared_error: 0.0964
Epoch 465/500
50

In [32]:
forward_model.save_weights("../data/model_stuff/forward_weights.h5")
binary_weights = forward_model.layers[0].get_weights()
inverse_weights = forward_model.layers[4].get_weights()

In [65]:
""" 
UNET MODEL
Fixing the weights for the bin_conv1 layer as well as the dense1 layer, ie NON TRAINABLE
Feeding in weights from the forward_model above to see if that improves the results from previous session

"""
inputs = Input(shape=(30,32,32,1),batch_size=50)
bin_conv1 = TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1',trainable=False))(inputs)
s = Lambda(streak, output_shape = streak_output_shape)(bin_conv1)
i = Lambda(integrate_ims, output_shape = integrate_ims_output_shape) (s)
f = Flatten()(i)
dense1 = Dense(30720, activation = 'relu',trainable=False)(f)
resh = Reshape((30,32,32,1))(dense1)
c1 = TimeDistributed(Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (resh)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
p1 = TimeDistributed(MaxPooling2D((2, 2)))(c1)

c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(p1)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
p2 = TimeDistributed(MaxPooling2D((2, 2)) )(c2)

c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p2)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
p3 = TimeDistributed(MaxPooling2D((2, 2)) )(c3)
    
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p3)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
p4 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (c4)

c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (p4)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)

u6 = TimeDistributed(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)

u7 = TimeDistributed(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
    
u8 = TimeDistributed(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)

u9 = TimeDistributed(Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET.compile(optimizer = Nadam(), loss = custom_loss, metrics = ['mean_squared_error'])

In [66]:
""" SET THE PREFIXED WEIGHTS FROM THE FORWARD MODEL """
CUPNET.layers[1].set_weights(binary_weights)
CUPNET.layers[5].set_weights(inverse_weights)
CUPNET.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(50, 30, 32, 32, 1) 0                                            
__________________________________________________________________________________________________
time_distributed_117 (TimeDistr (50, 30, 32, 32, 1)  1024        input_10[0][0]                   
__________________________________________________________________________________________________
lambda_18 (Lambda)              (50, 30, 62, 32, 1)  0           time_distributed_117[0][0]       
__________________________________________________________________________________________________
lambda_19 (Lambda)              (50, 62, 32, 1)      0           lambda_18[0][0]                  
____________________________________________________________________________________________

In [ ]:
""" CUPNET TRAINING """
CUPNET_history = CUPNET.fit(MX_train, My_train,
          batch_size = 50,epochs= 500,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr])



Train on 500 samples, validate on 250 samples
Epoch 1/500
500/500 - 15s - loss: 0.2424 - mean_squared_error: 0.0674 - val_loss: 0.2404 - val_mean_squared_error: 0.0690
Epoch 2/500
500/500 - 7s - loss: 0.2237 - mean_squared_error: 0.0592 - val_loss: 0.2268 - val_mean_squared_error: 0.0552
Epoch 3/500
500/500 - 7s - loss: 0.2068 - mean_squared_error: 0.0513 - val_loss: 0.2243 - val_mean_squared_error: 0.0533
Epoch 4/500
500/500 - 7s - loss: 0.1789 - mean_squared_error: 0.0309 - val_loss: 0.2018 - val_mean_squared_error: 0.0312
Epoch 5/500
500/500 - 7s - loss: 0.1680 - mean_squared_error: 0.0253 - val_loss: 0.1979 - val_mean_squared_error: 0.0292
Epoch 6/500
500/500 - 7s - loss: 0.1594 - mean_squared_error: 0.0211 - val_loss: 0.1947 - val_mean_squared_error: 0.0279
Epoch 7/500
500/500 - 7s - loss: 0.1589 - mean_squared_error: 0.0231 - val_loss: 0.1950 - val_mean_squared_error: 0.0292
Epoch 8/500
500/500 - 7s - loss: 0.1554 - mean_squared_error: 0.0212 - val_loss: 0.1956 - val_mean_squared

In [ ]:
CUPNET.save_weights("../data/model_stuff/cupnet_weights_equal_weighting.h5")

In [ ]:
y_pred = CUPNET.predict(MX_train)

In [ ]:
show_video(y_pred,My_train,30)

In [ ]:
fig,ax = plt.subplots(1,1)
ax.imshow(np.reshape(ims2,(-1,62,32))[30],cmap="gray")

In [ ]:
fig1,ax1 = plt.subplots(1,1)
ax1.imshow(np.reshape(validate2,(-1,30,32,32))[50][10],cmap="gray")

In [ ]:
fig2,ax2=plt.subplots(1,1)
ax2.imshow(np.reshape(validate,(-1,30,32,32))[50][10],cmap="gray")